In [ ]:
## このプログラムは「 Digit Recognizer」
## において作成したノートブックにおいて動作します

# Hyperasをインストール
!pip install hyperas

In [ ]:
# Hyperasはここでインポートする
from hyperopt import hp
from hyperopt import Trials, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

def prepare_data():
    """データを用意する。
    
    """
    # 関数で使用する外部ライブラリ
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import KFold
    ## keras modules
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, Flatten # core layers
    from tensorflow.keras.layers import Conv2D, MaxPooling2D    # convolution layers

    # train.csvを読み込んでpandasのDataFrameに格納
    train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
    train_x = train.drop(['label'], axis=1) # trainから画像データを抽出
    train_y = train['label']                # trainから正解ラベルを抽出
    test_x = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

    # trainのデータを学習データとテストデータに分ける。
    kf = KFold(n_splits=4, shuffle=True, random_state=123)
    tr_idx, va_idx = list(kf.split(train_x))[0]
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # 画像のピクセル値を255.0で割って0～1.0の範囲にしてnumpy.arrayに変換
    tr_x, va_x = np.array(tr_x / 255.0), np.array(va_x / 255.0)

    # 画像データの2階テンソルを
    # (高さ = 28px, 幅 = 28px , チャンネル = 1)の
    # 3階テンソルに変換
    # グレースケールのためチャンネルは1。
    tr_x = tr_x.reshape(-1,28,28,1)
    va_x = va_x.reshape(-1,28,28,1)
    
    # 正解ラベルをone-hot表現に変換
    tr_y = to_categorical(tr_y, 10)
    va_y = to_categorical(va_y, 10)

    return tr_x, tr_y, va_x, va_y

def create_model(tr_x, tr_y):
    """モデルを生成する。
    
    """
    # Sequentialオブジェクトを生成
    model = Sequential()

    # 第1層のフィルター数、フィルターのサイズを探索
    model.add(Conv2D(filters={{choice([32, 64])}},
                     kernel_size={{choice([(3,3), (5,5), (7,7)])}},
                     padding='same', 
                     activation={{choice(['tanh', 'relu'])}},
                     input_shape=(28,28,1)))

    # 第2層のフィルター数、フィルターのサイズを探索
    model.add(Conv2D(filters = {{choice([32, 64])}},
                     kernel_size = {{choice([(3,3), (5,5), (7,7)])}},
                     padding='same', 
                     activation={{choice(['tanh', 'relu'])}}
                    ))
    
    # 第3層に2×2のプーリング層を配置。
    model.add(MaxPooling2D(pool_size=(2,2)))

    # ドロップアウト率を探索。
    model.add(Dropout(
        {{quniform(0.2, 0.6, 0.05)}}
        ))
    

    # 第4層のフィルター数、フィルターのサイズを探索
    model.add(Conv2D(filters={{choice([32, 64])}},
                     kernel_size={{choice([(3,3), (5,5), (7,7)])}},
                     padding='same', 
                     activation='relu'))
    
    # 第5層のフィルター数、フィルターのサイズを探索
    model.add(Conv2D(filters = {{choice([32, 64])}},
                     kernel_size = {{choice([(3,3), (5,5), (7,7)])}},
                     padding='same', 
                     activation={{choice(['tanh', 'relu'])}}
                    ))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    # ドロップアウト率を探索。
    model.add(Dropout(
        {{quniform(0.2, 0.6, 0.05)}}
        ))

    # Flatten層を配置。
    model.add(Flatten())
    
    # 追加する層の数を1.2の中から探索。        
    if {{choice(['one', 'two'])}} == 'one':
        """ oneが選択されたら第6層を配置してユニット数、
            活性化関数、ドロップアウト率を探索する
        """
        # 第6層
        model.add(Dense(
            {{choice([500, 600, 700])}},
        activation={{choice(['tanh', 'relu'])}}
        ))
        model.add(Dropout(
            {{quniform(0.1, 0.6, 0.05)}}
        ))
       
    elif {{choice(['one', 'two'])}} == 'two':
        """ twoが選択されたら第6層、第7層を配置してユニット数、
            活性化関数、ドロップアウト率を探索する
        """
        # 第6層
        model.add(Dense(
            {{choice([500, 600, 700])}},
        activation={{choice(['tanh', 'relu'])}}
        ))
        model.add(Dropout(
            {{quniform(0.1, 0.6, 0.05)}}
        ))
        
        # 第7層
        model.add(Dense(
            {{choice([100, 150, 200])}},
        activation={{choice(['tanh', 'relu'])}}
        ))
        model.add(Dropout(
            {{quniform(0.2, 0.6, 0.05)}}
        ))

    # 出力層を配置。
    model.add(Dense(10, activation = "softmax"))

    # モデルのコンパイル。
    # オプティマイザーをAdamとRMSpropで試す
    model.compile(loss="categorical_crossentropy",
                  optimizer={{choice(['adam', 'rmsprop'])}},
                  metrics=["accuracy"])

    # 学習回数は30回
    epoch = 30
    # ミニバッチのサイズを100と200で試す。
    batch_size = {{choice([100, 200, 300])}}
    result = model.fit(tr_x, tr_y,
                       epochs=epoch,
                       batch_size=batch_size,
                       validation_data=(va_x, va_y),
                       verbose=0)

    # 簡易的に訓練時の結果を出力する
    validation_acc = np.amax(result.history['val_accuracy']) 
    print('Best validation acc of epoch:', validation_acc)
    
    # validation_accの値を最小化するように探索する
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

# 探索の実行。試行回数は100とする
best_run, best_model = optim.minimize(model=create_model,
                                      data=prepare_data,
                                      algo=tpe.suggest,
                                      max_evals=100,
                                      eval_space=True,
                                      notebook_name='__notebook_source__',
                                      trials=Trials())

In [ ]:
# 最も精度が優れていたモデルを出力。
print(best_model.summary())
# 最も精度が優れていたパラメーター値を出力。
print(best_run)

# 探索したモデルでテストデータを検証する。
_, _, va_x, va_y = prepare_data()
val_loss, val_acc = best_model.evaluate(va_x, va_y)
print("val_loss: ", val_loss) # 損失を出力。
print("val_acc: ", val_acc)   # 精度を出力。